In [ ]:
!pip install -q transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("glue", "sst2")

print("نمونه‌ای از داده آموزشی:")
print(dataset["train"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

نمونه‌ای از داده آموزشی:
{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"

# بارگذاری توکنایزر برای تبدیل متن به توکن
tokenizer = AutoTokenizer.from_pretrained(model_name)

# بارگذاری مدل برای طبقه‌بندی متن با دو لیبل (مثبت/منفی)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    """این تابع متن‌ها را به فرمت ورودی مدل تبدیل می‌کند."""
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

# اعمال تابع روی تمام بخش‌های داده
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
import evaluate

# بارگذاری معیار دقت
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """این تابع دقت را در حین ارزیابی محاسبه می‌کند."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

# تعریف آرگومان‌های آموزش
training_args = TrainingArguments(
    output_dir="distilbert-finetuned-sst2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)


# ساخت Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# شروع فرآیند آموزش
trainer.train()

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
!pip install --upgrade transformers datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 26.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.5
    Uninstalling huggingface-hub-0.33.5:
      Successfully uninstalled huggingface-hub-0.33.5
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [ ]:
import transformers
print(transformers.__version__)  # should be ≥4.3.0


4.54.0


In [ ]:
from transformers import TrainingArguments, Trainer

# تعریف آرگومان‌های آموزش
training_args = TrainingArguments(
    output_dir="distilbert-finetuned-sst2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)


# ساخت Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# شروع فرآیند آموزش
trainer.train()

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
from transformers import TrainingArguments, Trainer

# تعریف آرگومان‌های آموزش با سینتکس قدیمی
training_args = TrainingArguments(
    output_dir="distilbert-finetuned-sst2",
    # خط زیر جایگزین شده است
    evaluate_during_training=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # این دو خط را می‌توانید حذف کنید یا نگه دارید
    # load_best_model_at_end=True,
    # metric_for_best_model="accuracy",
)

# ساخت Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# شروع فرآیند آموزش
trainer.train()

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluate_during_training'

In [ ]:
from transformers import TrainingArguments, Trainer

# تعریف آرگومان‌های آموزش با سینتکس صحیح و مدرن
training_args = TrainingArguments(
    output_dir="distilbert-finetuned-sst2",
    evaluation_strategy="epoch",  # از این آرگومان استفاده کنید
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# ساخت Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# شروع فرآیند آموزش
trainer.train()

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
# اول بررسی نسخه transformers
import transformers
print(f"نسخه transformers: {transformers.__version__}")

# کد سازگار با نسخه‌های مختلف
from transformers import TrainingArguments, Trainer
import sys

# تشخیص نسخه و استفاده از پارامترهای مناسب
def create_training_args():
    version = transformers.__version__
    major_version = int(version.split('.')[0])

    if major_version >= 4 and int(version.split('.')[1]) >= 11:
        # نسخه‌های جدید (4.11+)
        return TrainingArguments(
            output_dir="distilbert-finetuned-sst2",
            eval_strategy="epoch",        # جایگزین evaluation_strategy
            save_strategy="epoch",
            num_train_epochs=3,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            logging_dir="./logs",
            logging_steps=100,
            report_to=[],  # غیرفعال کردن wandb
        )
    elif major_version >= 4:
        # نسخه‌های متوسط (4.0-4.10)
        return TrainingArguments(
            output_dir="distilbert-finetuned-sst2",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            num_train_epochs=3,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            logging_dir="./logs",
            logging_steps=100,
        )
    else:
        # نسخه‌های قدیمی (3.x)
        return TrainingArguments(
            output_dir="distilbert-finetuned-sst2",
            evaluate_during_training=True,
            save_steps=500,
            eval_steps=500,
            num_train_epochs=3,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            load_best_model_at_end=True,
            logging_steps=100,
        )

try:
    # تلاش برای ساخت training_args
    training_args = create_training_args()
    print("تنظیمات آموزش با موفقیت ایجاد شد")

except Exception as e:
    print(f"خطا در ساخت training_args: {e}")
    # استفاده از تنظیمات حداقلی
    training_args = TrainingArguments(
        output_dir="distilbert-finetuned-sst2",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
    )
    print("از تنظیمات حداقلی استفاده شد")

# ساخت Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

print("شروع آموزش...")
trainer.train()

# نمایش نتایج ارزیابی
eval_results = trainer.evaluate()
print(f"نتایج ارزیابی: {eval_results}")

# ذخیره مدل
trainer.save_model("./final_model")
print("مدل ذخیره شد در ./final_model")

نسخه transformers: 4.54.0
تنظیمات آموزش با موفقیت ایجاد شد


NameError: name 'model' is not defined

In [ ]:
# راه‌حل ساده - فقط پارامتر را تغییر دهید
from transformers import TrainingArguments, Trainer

# تعریف آرگومان‌های آموزش - نسخه اصلاح‌شده
training_args = TrainingArguments(
    output_dir="distilbert-finetuned-sst2",
    eval_strategy="epoch",        # تغییر از evaluation_strategy به eval_strategy
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# اگر بالا کار نکرد، از این استفاده کنید:
# training_args = TrainingArguments(
#     output_dir="distilbert-finetuned-sst2",
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
# )

# ساخت Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# شروع فرآیند آموزش
trainer.train()

NameError: name 'model' is not defined

In [ ]:
# گام ۱: نصب و وارد کردن کتابخانه‌ها
!pip install -q transformers datasets evaluate accelerate
import transformers
from datasets import load_dataset
import numpy as np
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

print(f"نسخه transformers: {transformers.__version__}")

# ------------------------------------------------------------------
# گام ۲: بارگذاری مدل و توکنایزر (این بخش در کد شما نبود)
# ------------------------------------------------------------------
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
print(">>> مدل و توکنایزر با موفقیت بارگذاری شد.")


# ------------------------------------------------------------------
# گام ۳: بارگذاری و پیش‌پردازش داده (این بخش در کد شما نبود)
# ------------------------------------------------------------------
dataset = load_dataset("glue", "sst2")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(">>> مجموعه داده بارگذاری و پیش‌پردازش شد.")


# ------------------------------------------------------------------
# گام ۴: تعریف معیار ارزیابی (این بخش در کد شما نبود)
# ------------------------------------------------------------------
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

print(">>> تابع محاسبه معیار ارزیابی تعریف شد.")


# ------------------------------------------------------------------
# گام ۵: کد شما برای تعریف آرگومان‌های آموزش (سازگار با نسخه‌های مختلف)
# ------------------------------------------------------------------
def create_training_args():
    version = transformers.__version__
    major_version = int(version.split('.')[0])

    # برای سادگی، فقط از سینتکس مدرن استفاده می‌کنیم چون کتابخانه آپدیت شده است
    return TrainingArguments(
        output_dir="distilbert-finetuned-sst2",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        report_to=[], # غیرفعال کردن لاگ‌های اضافی مانند wandb
    )

training_args = create_training_args()
print(">>> تنظیمات آموزش با موفقیت ایجاد شد.")


# ------------------------------------------------------------------
# گام ۶: ساخت Trainer و شروع آموزش
# ------------------------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

print("\n>>> شروع فرآیند آموزش...")
trainer.train()

# نمایش نتایج
print("\n>>> نتایج ارزیابی نهایی:")
eval_results = trainer.evaluate()
print(eval_results)

نسخه transformers: 4.54.0


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


>>> مدل و توکنایزر با موفقیت بارگذاری شد.


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

>>> مجموعه داده بارگذاری و پیش‌پردازش شد.


>>> تابع محاسبه معیار ارزیابی تعریف شد.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
# گام ۱: نصب و وارد کردن کتابخانه‌ها
!pip install -q transformers datasets evaluate accelerate
import transformers
from datasets import load_dataset
import numpy as np
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

print(f"نسخه transformers: {transformers.__version__}")

# ------------------------------------------------------------------
# گام ۲: بارگذاری مدل و توکنایزر (این بخش در کد شما نبود)
# ------------------------------------------------------------------
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
print(">>> مدل و توکنایزر با موفقیت بارگذاری شد.")


# ------------------------------------------------------------------
# گام ۳: بارگذاری و پیش‌پردازش داده (این بخش در کد شما نبود)
# ------------------------------------------------------------------
dataset = load_dataset("glue", "sst2")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(">>> مجموعه داده بارگذاری و پیش‌پردازش شد.")


# ------------------------------------------------------------------
# گام ۴: تعریف معیار ارزیابی (این بخش در کد شما نبود)
# ------------------------------------------------------------------
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

print(">>> تابع محاسبه معیار ارزیابی تعریف شد.")


# ------------------------------------------------------------------
# گام ۵: کد شما برای تعریف آرگومان‌های آموزش (سازگار با نسخه‌های مختلف)
# ------------------------------------------------------------------
def create_training_args():
    version = transformers.__version__
    major_version = int(version.split('.')[0])

    # برای سادگی، فقط از سینتکس مدرن استفاده می‌کنیم چون کتابخانه آپدیت شده است
    return TrainingArguments(
        output_dir="distilbert-finetuned-sst2",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        report_to=[], # غیرفعال کردن لاگ‌های اضافی مانند wandb
    )

training_args = create_training_args()
print(">>> تنظیمات آموزش با موفقیت ایجاد شد.")


# ------------------------------------------------------------------
# گام ۶: ساخت Trainer و شروع آموزش
# ------------------------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

print("\n>>> شروع فرآیند آموزش...")
trainer.train()

# نمایش نتایج
print("\n>>> نتایج ارزیابی نهایی:")
eval_results = trainer.evaluate()
print(eval_results)

نسخه transformers: 4.54.0


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


>>> مدل و توکنایزر با موفقیت بارگذاری شد.


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

>>> مجموعه داده بارگذاری و پیش‌پردازش شد.
>>> تابع محاسبه معیار ارزیابی تعریف شد.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
!pip install --upgrade transformers



In [ ]:
import transformers, inspect
print(transformers.__version__)   # می‌بایست >=4.3.0 باشد
print(inspect.signature(transformers.TrainingArguments.__init__))


4.54.0
(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict[str, Any], str, NoneType] = <factory>, warmup_ratio: float 

In [ ]:
# گام ۱: نصب و وارد کردن کتابخانه‌ها
!pip install -q transformers datasets evaluate accelerate
import transformers
from datasets import load_dataset
import numpy as np
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

print(f"نسخه transformers: {transformers.__version__}")

# ------------------------------------------------------------------
# گام ۲: بارگذاری مدل و توکنایزر
# ------------------------------------------------------------------
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
print(">>> مدل و توکنایزر با موفقیت بارگذاری شد.")

# ------------------------------------------------------------------
# گام ۳: بارگذاری و پیش‌پردازش داده
# ------------------------------------------------------------------
dataset = load_dataset("glue", "sst2")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(">>> مجموعه داده بارگذاری و پیش‌پردازش شد.")

# ------------------------------------------------------------------
# گام ۴: تعریف معیار ارزیابی
# ------------------------------------------------------------------
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

print(">>> تابع محاسبه معیار ارزیابی تعریف شد.")

# ------------------------------------------------------------------
# گام ۵: تعریف آرگومان‌های آموزش (سازگار با نسخه ۴.۵۴.۰)
# ------------------------------------------------------------------
def create_training_args():
    r


نسخه transformers: 4.54.0


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


>>> مدل و توکنایزر با موفقیت بارگذاری شد.


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Exception ignored in: <function _xla_gc_callback at 0x7c38c3496c00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
# گام ۱: نصب و وارد کردن کتابخانه‌ها
!pip install -q transformers datasets evaluate accelerate
import transformers
from datasets import load_dataset
import numpy as np
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

print(f"نسخه transformers: {transformers.__version__}")

# ------------------------------------------------------------------
# گام ۲: بارگذاری مدل و توکنایزر
# ------------------------------------------------------------------
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
print(">>> مدل و توکنایزر با موفقیت بارگذاری شد.")

# ------------------------------------------------------------------
# گام ۳: بارگذاری و پیش‌پردازش داده
# ------------------------------------------------------------------
dataset = load_dataset("glue", "sst2")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(">>> مجموعه داده بارگذاری و پیش‌پردازش شد.")

# ------------------------------------------------------------------
# گام ۴: تعریف معیار ارزیابی
# ------------------------------------------------------------------
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

print(">>> تابع محاسبه معیار ارزیابی تعریف شد.")

# ------------------------------------------------------------------
# گام ۵: تعریف آرگومان‌های آموزش (سازگار با نسخه ۴.۵۴.۰)
# ------------------------------------------------------------------
def create_training_args():
    return TrainingArguments(
        output_dir="distilbert-finetuned-sst2",
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        eval_strategy="epoch",        # به‌جای evaluation_strategy
        save_strategy="epoch",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        report_to=[],                 # غیرفعال کردن لاگ‌های اضافی
    )

training_args = create_training_args()
print(">>> تنظیمات آموزش با موفقیت ایجاد شد.")

# ------------------------------------------------------------------
# گام ۶: ساخت Trainer و شروع آموزش
# ------------------------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

print("\n>>> شروع فرآیند آموزش...")
trainer.train()

# نمایش نتایج
print("\n>>> نتایج ارزیابی نهایی:")
eval_results = trainer.evaluate()
print(eval_results)


نسخه transformers: 4.54.0


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


>>> مدل و توکنایزر با موفقیت بارگذاری شد.


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

>>> مجموعه داده بارگذاری و پیش‌پردازش شد.
>>> تابع محاسبه معیار ارزیابی تعریف شد.
>>> تنظیمات آموزش با موفقیت ایجاد شد.

>>> شروع فرآیند آموزش...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.184900,0.329494,0.891055
2,0.129000,0.415583,0.895642
3,0.064300,0.432642,0.899083



>>> نتایج ارزیابی نهایی:


{'eval_loss': 0.4326421022415161, 'eval_accuracy': 0.8990825688073395, 'eval_runtime': 13.4434, 'eval_samples_per_second': 64.864, 'eval_steps_per_second': 4.091, 'epoch': 3.0}


In [ ]:
# ------------------------------------------------------------------
# Step 7: Final evaluation and testing on new sentences
# ------------------------------------------------------------------

# Final evaluation on the validation dataset
print("\n>>> Final evaluation results on the validation dataset:")
eval_results = trainer.evaluate()
print(eval_results)

# Compare with the paper
print(f"\nComparison with the paper: Our model accuracy: {eval_results['eval_accuracy']:.4f} | Paper accuracy: 0.913")

# Use pipeline for easy testing of the model
from transformers import pipeline

# Create a pipeline with your fine-tuned model
sentiment_pipeline = pipeline(
    "sentiment-analysis",  # task name
    model=model,           # fine-tuned model
    tokenizer=tokenizer,
    device=0               # run on GPU if available
)

# Test the model with custom sentences
custom_sentences = [
    "I love how simple and efficient this is!",
    "This movie was a complete waste of my time.",
    "The story was good, but the acting was not impressive.",
]

print("\n>>> Testing the model on new sentences:")
results = sentiment_pipeline(custom_sentences)

for sentence, result in zip(custom_sentences, results):
    # Labels in this dataset: LABEL_1 (positive), LABEL_0 (negative)
    label = "Positive" if result['label'] == 'LABEL_1' else "Negative"
    print(f'Sentence: "{sentence}"')
    print(f'  -> Detected sentiment: {label} (Score: {result["score"]:.4f})\n')

# Save the final model and tokenizer
final_model_path = "./distilbert-final-model"
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"The final model and tokenizer have been saved to '{final_model_path}'.")



>>> Final evaluation results on the validation dataset:


Device set to use cuda:0


{'eval_loss': 0.4326421022415161, 'eval_accuracy': 0.8990825688073395, 'eval_runtime': 12.8644, 'eval_samples_per_second': 67.784, 'eval_steps_per_second': 4.275, 'epoch': 3.0}

Comparison with the paper: Our model accuracy: 0.8991 | Paper accuracy: 0.913

>>> Testing the model on new sentences:
Sentence: "I love how simple and efficient this is!"
  -> Detected sentiment: Positive (Score: 0.9999)

Sentence: "This movie was a complete waste of my time."
  -> Detected sentiment: Negative (Score: 0.9997)

Sentence: "The story was good, but the acting was not impressive."
  -> Detected sentiment: Negative (Score: 0.9984)

The final model and tokenizer have been saved to './distilbert-final-model'.
